In [ ]:
import pandas as pd
import numpy as np
import scipy
import pylab as pl
import sklearn as sk
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_excel('HealthViz County Dataset 6.19.17.xlsx',skiprows=0, header=1, index_col=0)
data.index.name=None

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:

def cov(x,y,w=None):
    '''
    Calculates covariance of x,y weighted by w.
    Parameters
        x,y: pd.Series
        w: pd.Series or None
    Returns
        covariance: float
    '''
    if w is None:
        w = pd.Series(np.ones(x.shape[0]))
        w.index = x.index
        
    numerator = sum(w*(x - np.average(x, weights=w))*(y - np.average(y, weights=w)))
    denominator = sum(w)
    return numerator/denominator

def corr(x, y, w=None, check_nulls = False):
    '''
    Calculates Pearson correlation between x,y weighted by w.
    Parameters
        x,y: pd.Series
        w: pd.Series or None
        check_nulls: Boolean, optional 
    Returns
        covariance: float
    '''
    if w is None:
        w = pd.Series(np.ones(x.shape[0]))
        w.index = x.index
    
    if check_nulls:
        df = pd.concat([x, y, w], axis = 1)
        null_cols  = df.isnull().any(axis = 1)
    
        if null_cols.any():
            x = x[~null_cols]
            y = y[~null_cols]
            w = w[~null_cols]

    return cov(x,y,w)/np.sqrt(cov(x,x,w)*cov(y,y,w))



In [ ]:
def df_corr(xs, ys, w=None, method=corr, min_periods=1):
        """
        Compute pairwise correlation of columns, excluding NA/null values
        Parameters
        ----------
        min_periods : int, optional
            Minimum number of observations required per pair of columns
            to have a valid result. 
        Returns
        -------
        y : DataFrame
        
        Modified from pandas:
            https://github.com/pandas-dev/pandas/blob/v0.20.2/pandas/core/frame.py#L4817-L4871
        """
        if isinstance(xs, pd.Series):
            xs = pd.DataFrame(xs)
            
        rows = xs.columns
       
        if isinstance(ys, pd.Series):
            ys = pd.DataFrame(ys)
            
        cols = ys.columns
            
        xmat = xs.values.T
        ymat = ys.values.T

        if min_periods is None:
            min_periods = 1
        #mat = _ensure_float64(mat).T
        corrf = method
        K = len(rows)
        M = len(cols)
        correl = np.empty((K, M), dtype=float)
        xmask = np.isfinite(xmat)
        ymask = np.isfinite(ymat)
        for i, ac in enumerate(xmat):
            for j, bc in enumerate(ymat):
                valid = xmask[i] & ymask[j]
                
                if valid.sum() < min_periods:
                    c = NA
                elif not valid.all():
                    c = corrf(ac[valid], bc[valid], w[valid])
                else:
                    c = corrf(ac, bc, w)
                correl[i, j] = c

        return pd.DataFrame(correl, index=rows, columns=cols)

In [ ]:
w = data.ix[:,1]
xs = data.ix[:,2:27]
ys = data.ix[:,27:-1]
crs = df_corr(xs,ys,w)
sns.heatmap(crs)

In [ ]:
crs

In [ ]:
def ordered_corrs(xs, y, w):
    '''
    '''
    crs = df_corr(xs,y,w)
    return crs.reindex(crs.ix[:,0].abs().sort_values(ascending=False).index).ix[:,0]

def ordered_heatmap(crs):
    pl.figure()
    sns.heatmap(pd.DataFrame(crs), vmin = -.8, vmax = .8)
    pl.show()

In [ ]:
for y in ys:
    ordered_heatmap(ordered_corrs(xs,ys[y], w))


In [ ]:
from sklearn import linear_model
import statsmodels.formula.api as smf

In [ ]:
def drop_nan(X,y, sample_weight=None, drop_threshold = .1, verbose = False):
    '''
    
    drop_threshold: float (between 0 and 1)
        When the proportion of nans in a column is above threshold, drop the column
    '''
    
    # Drop all data without an outcome measure
    valid_y = np.isfinite(y) 
    if verbose:
        dropped = valid_y.count() - valid_y.sum()
        print("Dropped {} rows with nan in outcome variable y\n".format(dropped))
    X = X[valid_y]
    y = y[valid_y]
    
    
    # Drop columns where the proportion of nans is above the drop_threshold
    drop_columns = X.isnull().sum() / X.shape[0] > drop_threshold
    if verbose:
        dropped = X.columns[drop_columns]
        message = "Dropped columns: "
        for d in dropped:
            message += d + "\n"
        print(message)      
    passable_columns = X.columns[~drop_columns]
    X = X[passable_columns]
    
    
    # Drop rows where we find NaNs in the Xs.
    xmask = np.isfinite(X)
    valid_X = xmask.all(axis=1)
    if verbose:
        dropped = valid_X.count() - valid_X.sum()
        print("Dropped {} rows with nans in X".format(dropped))
    X = X[valid_X]
    y = y[valid_X]
    
    if sample_weight is not None:
        sample_weight = sample_weight[valid_y][valid_X]
        return X, y, sample_weight
    
    return X, y

In [ ]:
X = xs

for n in range(ys.shape[1]):
    y = ys.ix[:,n]
    a,b,wgt = drop_nan(X,y, sample_weight=w)
    lm = linear_model.LinearRegression()
    model = lm.fit(a,b, sample_weight=wgt)
    score = lm.score(a,b, sample_weight=wgt)
    print("R^2 for {var}: {score}".format(var=ys.columns[n], score=score))

In [ ]:
coef= pd.DataFrame(model.coef_, index = a.columns, columns = [b.name])
coef.sort_values(b.name)

In [ ]:
def ols(X,y, drop_threshold = .1, verbose = False):
    X,y = drop_nan(X,y, verbose=True)
    lm = linear_model.LinearRegression()
    model = lm.fit(X,y)
    coef= pd.DataFrame(model.coef_, index = a.columns, columns = [b.name])
    return coef.sort_values(b.name)

# next steps do we 

In [ ]:
a = np.array([[1,1,1,0,0,0],[0,0,0,1,1,1]]).T
b = np.array([.3, .4, .8, .1, .2, .4])



In [ ]:
lm = linear_model.LinearRegression()
model = lm.fit(a,b)

In [ ]:
model.__dict__

In [ ]:
import statsmodels
data = pd.read_excel('HealthViz County Dataset',skiprows=0, header=1, index_col=0)
data.index.name=None

In [ ]:
results = sm.OLS(a, b).fit()


In [398]:
weights = df['Population (residents), 2011-2015']

y = df['Diabetes mortality (deaths per 100,000), 2008-2014']

#var = ['Election margin, winner (Presidential) (% margin), 2016']
controls = ['Population, Non-Hispanic Black (residents), 2011-2015',
'Population, Hispanic or Latino (residents), 2011-2015',
'Population, Asian or Pacific Islander (residents), 2011-2015',
'Median age, 2011-2015',
'Median household income, 2011-2015',
'Poverty rate (% of residents), 2011-2015',
'Share of income, top 5% (% of total income), 2011-2015',
'Dual eligible coverage (% of residents), 2010-2014',
'College graduation rate (% of residents), 2011-2015',
'Graduate education rate (% of residents), 2011-2015',
'Food stamps (SNAP) (% of households), 2011-2015']


#X = df[var + controls]

data_var = 'Readmission rate (Medicare) (% of hospital admissions), 2015'
X = pd.merge(pd.DataFrame(data[data_var]), df[controls],right_index=True, left_index=True )

X,y,w = drop_nan(X,y, weights, verbose = True)

In [398]:
X_1 = sm.add_constant(X)
model = sm.OLS(y,X_1)
model2 = sm.WLS(y,X_1,weights=w)
model.fit().summary()
#model2.fit().summary()
results = model.fit()
betas = results.params.ix['Readmission rate (Medicare) (% of hospital admissions), 2015'])
hetero_se = results.HC0_se.ix['Readmission rate (Medicare) (% of hospital admissions), 2015']